In [ ]:
# Load google drive
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/LogoDetector/
!make all

# Prepare Open Set Documents and Append New Classes

Use prepare_openset.py to prepare the files for open set classification (in detect.py) and thresholds calculation (in thresholds.py). This file can also be used to append classes to existing lookup tables. See arguments by:

```bash
python prepare_openset.py -h
```

Important arguments and details are listed below.

## Classifier

Arguments:
1. --classifier
2. --classifier-weights
3. --n-classes
4. --open-type

## Creating Lookup Table from Scratch

Arguments:
1. -t, --train

## Using Pre-Existing Lookup Table

Arguments:
1. --train-avs
2. --train-labs
3. --train-class-dict

## Appending Classes to Pre-Existing Lookup Table

The user needs to only add the new classes (folders) to their existing train dataset.

Arguments:
1. -a, --append-classes

## Creating Open Set Folders

Each folder contains the true labels of the input data, the predicted labels (nearest K), and the distances (nearest K) from the lookup table used to assign the predicted labels (nearest K).

Arguments:
1. -n, --name
2. -s, --source
3. --K (Optional)



In [ ]:
!(python prepare_openset.py -t PATH_TO_TRAIN_DATA \
-n known -s PATH_TO_HELD_OUT_KNOWN \
-n unknown -s PATH_TO_HELD_OUT_UNKNOWN \
--project PROJECT_NAME \
--open-type OPENSET_TYPE \
--classifier CLASSIFIER_NAME \
--classifier-weights PATH_TO_CLASSIFIER_WEIGHTS \
--n-classes NUM_CLASSES_USED_TO_TRAIN_CLASSIFIER)

Namespace(K=2, append_classes=False, classifier='efficientnet_b3', classifier_weights='weights/model-b3-closed-decay-ar-aug3.pt', device='', n_classes=32, name=['known', 'unknown'], open_type=2, project='test', source=['/content/drive/.shortcut-targets-by-id/1xJwN_3Fb5GrUvXR60GCXOAUjZob0ggt1/clean_logos/output/closed_set_val', '/content/drive/.shortcut-targets-by-id/1xJwN_3Fb5GrUvXR60GCXOAUjZob0ggt1/clean_logos/output/unknown_train'], train='/content/drive/.shortcut-targets-by-id/1xJwN_3Fb5GrUvXR60GCXOAUjZob0ggt1/clean_logos/output/closed_set_train', train_avs=None, train_class_dict=None, train_labs=None)

Loading in data from /content/drive/.shortcut-targets-by-id/1xJwN_3Fb5GrUvXR60GCXOAUjZob0ggt1/clean_logos/output/closed_set_train... DONE

Calculating train activation vectors... DONE (87.38s)

True labels, predicted labels (top 2), associated distances, and activation vectors to config/activations/test/train/
Loading in data from /content/drive/.shortcut-targets-by-id/1xJwN_3Fb5GrUv

# Obtaining Thresholds and Testing Thresholds

Use thresholds.py to obtain thresholds (--T in detect.py) for open set classification. This file can also be used to test the performance of calculated or provided thresholds. See arguments by:

```bash
python threshold.py -h
```

Important arguments and details are listed below.

## Calculate Thresholds

Arguments:
1. --name
2. --train (folder generated by prepare_openset.py)
3. --known (folder generated by prepare_openset.py)
4. --unknown (folder generated by prepare_openset.py)

## Test Thresholds

Arguments:
1. --T (Optional)
2. --test (folder generated by prepare_openset.py)
3. --test-unknown (folder generated by prepare_openset.py)


In [ ]:
!(python thresholds.py --name NAME_OF_THRESHOLDS_FILE \
--train PATH_TO_OPENSET_TRAIN_FOLDER \
--known PATH_TO_OPENSET_KNOWN_FOLDER \
--unknown PATH_TO_OPENSET_UNKNOWN_FOLDER)

Namespace(T=None, device='', known='config/activations/test/known/', name='Ts', test=None, test_unknown=None, train='config/activations/test/train/', unknown='config/activations/test/unknown/')

Calculating Thresholds... DONE (4.44s)
Thresholds saved to config/Ts.p

TRAIN PERFORMANCE
Correctly Identified Unknown Percentage (Openset Accuracy): 95.57%
Incorrectly Called Unknown: 4.09%
Micro Top 1 Accuracy: 95.19%



# Detection and Open Set Classification

Use detect.py for detection and open set classification. See arguments by:

```bash
python detect.py -h
```

Important arguments and details are listed below.

## Detectors

Arguments:
1. -d, --detector
2. -dw, --detector-weights
3. -dt, --detector-threshold
4. --rcnn-arch 

Users can have a maximum of two -d, -dw, and -dt arguments. Having only one argument for each means the user only uses that detector. If there are two arguments, the detectors are __ensembled__.

## Classifier

Arguments:
1. -c, --classifier 
2. -cw, --classifier-weights
3. --brand-mapping (output from prepare_openset.py: train_class_dict.p)
4. --n-classes

## Open Set Learning

Arguments:
1. --open-type 
2. --T (output from thresholds.py)
3. --train-avs (output from prepare_openset.py: train_avs.pt)
4. --train-labs (output from prepare_openset.py: train_labs.pt)

## Images

Arguments:
1. --source

## Output

Arguments:
1. --save-json
2. --save-img
3. --name (optional)
4. --project (optional)

__Note__: The other arguments to this file are completely optional and their functions can be read/understood by visiting https://github.com/ultralytics/yolov3.

In [ ]:
!(python detect.py -d NAME_OF_DETECTOR \
-dw PATH_TO_DETECTOR_WEIGHTS \
-dt THRESHOLD_FOR_DETECTOR_WEIGHTS \
-c NAME_OF_CLASSIFIER \
-cw PATH_TO_CLASSIFIER_WEIGHTS \
--source PATH_TO_IMAGES_FOR_PREDICTIONS \
--name NAME_OF_FOLDER_TO_SAVE_RESULTS \
--train-avs PATH_TO_TRAIN_ACTIVATION_VECTORS \
--train-labs PATH_TO_TRAIN_LABELS \
--brand-mapping PATH_TO_TRAIN_CLASS_DICT \
--T PATH_TO_THRESHOLDS \
--n-classes NUM_CLASSES_USED_TO_TRAIN_CLASSIFIER \
--save-img \
--save-json) 

Namespace(T='config/Ts.p', agnostic_nms=False, augment=False, brand_mapping='config/activations/test/train/train_class_dict.p', classes=None, classifier='efficientnet_b3', classifier_weights='weights/model-b3-closed-decay-ar-aug3.pt', detect_thres=['0.3'], detector=['yolo'], detector_weights=['weights/yolo.pt'], device='', exist_ok=False, img_size=2560, iou_thres=0.45, n_classes=32, name='experiment', open_type=2, project='runs/detect', rcnn_arch='COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', save_img=True, save_json=False, source='/content/drive/.shortcut-targets-by-id/1xJwN_3Fb5GrUvXR60GCXOAUjZob0ggt1/clean_logos/output/demo/', train_avs='config/activations/test/train/train_avs.pt', train_labs='config/activations/test/train/train_labs.pt', transform_sz=300)
YOLOv3  torch 1.7.1 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 269 layers, 62546518 parameters, 0 gradients, 155.7 GFLOPS
image 1/4 dc647a1bdd7c62902b9da767c1ead9cbbaeafeecb4e2728f001a8176a0d6a3ac (1).p